# k-Nearest Neighbour Search

Consider the problem of predicting a class label, for example a plant species, given a certain number of features that were recorded for the plant.
K-Nearest Neighbour (kNN) is a non-parametric machine learning method for predicting class labels by comparing test features with all training features.
The classifier first finds the k most similar training features and then returns the most common class label as the predicted label.

In this assignment we will work with the [Iris flower dataset](https://onlinelibrary.wiley.com/doi/10.1111/j.1469-1809.1936.tb02137.x).
This dataset classifies flowers into three different species `Versicolor`, `Virginica`, and `Setosa`.
For each flower instance, four different features are collected: 
1. sepal length (in cm)
1. sepal width (in cm)
1. petal length (in cm)
1. petal width (in cm)

In the following code cell, we first load all necessary imports and then load the Iris dataset from disk.
The training dataset is then logged to console as a Pandas dataframe.

> **_Important:_**
To complete this assignment, always use the file `data/iris.json` together with the provided class `IrisData`.
Do not download or use any other version of the Iris dataset.
Your work will only be evaluated against the dataset included in this mini project repository.

In [ ]:
import numpy as np
import os
import plotly.express as px
from p1.iris_knn import IrisData, confusion_matrix
from IPython.display import display, Markdown

iris_data = IrisData.from_json(os.path.join("data", "iris.json"))
iris_data.all

Each row in this dataset corresponds to a plant whose features (sepal length and width and petal length and width) have been recorded together with their species classification.
Each plant's four features can be concatenated into a four-dimensional vector $\pmb{x} \in \mathbb{R}^4$, where each entry of the vector $\pmb{x}$ corresponds to the sepal length, sepal width, petal length, and petal width.
Formally, each plant's features in this dataset can then be written as a pair $(\pmb{x}_i, l_i)$, where $\pmb{x}_i$ is the $i$ th plant's feature vector and $l_i$ is the associated species label.
In the dataframe above, plant species are represented as a string.
The class `IrisData` in the module `p1.iris_knn.py` can be used to map these strings to an array of species indices.
The method `IrisData.species_to_index` can be used to map species string to indices.
The dataset logged in the dataframe above can then be written as the set $$\mathcal{D} = \{ (\pmb{x}_i, l_i) | i = 1,...,D \}.$$

The modified Iris dataset provided in this project is split into a training set and a test set.
You can access this data through the properties of the `IrisData` class:

In [ ]:
print("Training feature vectors (first 3):")
iris_data.train_features[:3]

Check out the implementation of the `IrisData` class to see how to use the Iris dataset in this project.

The goal of this mini project is to implement the kNN method to predict the species of the test feature vectors using only the training feacture vectors and the training species labels.

## The k-nearest neighbour method

Suppose we are provided with an additional flower for which we want to determine the species.
This test flower has a feature vector $\pmb{x}_\text{test}$.
The k-nearest neighbour method then iterates through the training dataset $\mathcal{D}$ to find the $k$ closest points by computing the euclidean distance between $\pmb{x}_\text{test}$ and each training feature vector $\pmb{x}_i$.
Once the k closest training points are found (from the dataset $\mathcal{D}$), the class labels corresponding to these k nearest neighbours are analyzed.
The class label that is most common amongst these k neighbours is predicted as the correct label for our test plant with feature vector $\pmb{x}_\text{test}$.

The objective of this mini project is to implement k-nearest neighbour clustering using the provided code base.
To complete this mini project, work through the steps outlined in this notebook.

> **_Important:_**
Everytime you change code outside of this notebook, you need to reload the Jupyter engine.
When the Jupyter engine starts it loads all Python modules in its path into memory.
If you make a change in any of these modules, for example by implementing a function in `p1.iris_knn.py`, this module needs to be reloaded by restarting the Jupyter engine.
To restart the Jupyter engine in VSCode, hit the "Restart" button at the top.

## Step 1: Plot the dataset [5 points]

To test if the dataset is loading correctly, create a scatter matrix plot using the Plotly function [`px.scatter_matrix`](https://plotly.com/python/splom/).
Only include the four features in this matrix plot.
The dataset and feature columns must be loaded through the `IrisData` class which has properties such as `all` and `feature_column`.

In [ ]:
# Insert your code here.

## Step 2: kNN implementation

The kNN algorithm is implemented in the module `p1.iris_knn.py`. 
For completing this mini project, you only need to implement the some of the functions in this module and change this notebook.
In `test/test_iris_knn.py` you can find unit tests for each of the functions you need to implement.
These unit tests test each function using a small example.
These examples allow you to debug your code to ensure its correctness.

### Step 2.1: Distance matrix calculation [15 points]

Implement functions `distance_matrix_from_features` in the module `p1.iris_knn`.
This function computes a distance matrix $\pmb{D}$ between two sets of feature matrices $\pmb{X}_{\text{test}}$ and $\pmb{X}_{\text{train}}$.
Each row of these matrices is set to a feature vector of $d$ dimensions.
For the Iris dataset, $d = 4$, but the implementation of this function should work for any number of dimensions.
The entry $(i, j)$ (ith row and jth column) is set to the euclidean distance between the ith row in $\pmb{X}_{\text{test}}$ and the jth row in $\pmb{X}_{\text{train}}$.

The unit test for this function is implemented in `test_distance_matrix_from_features` in the file `test/test_iris_knn.py`.

To get started with a correct implementation, implement `distance_matrix_from_features` using Python for loops. 
Then, move towards an implementation with tensor arithmetic (without for loops or if-else statements).
For full marks, do not use Python for loops or if-else statements.

### Step 2.2: Find k nearest neighbours [15 points]

The distance matrix constructed in Step 2.1 stores information about the distances between each test feature vector and each lablled training feature vector.
To find the k nearest neighbours of each test feature vector, we need to search through each row of the distance matrix and find the column indices of the lowest k elements.
These column indices correspond to the row vectors in `IrisData.train_features` that are the closest k neighbours.
We need these indices to look up the species indices stored in `IrisData.train_species_idxs`.
This operation of finding the indices of the k nearest training feature vectors should be implemented in the `distance_matrix_to_knn_indices` function. 

To debug your implementation, you should use the unit test `test_distance_matrix_to_knn_indices` in the file `test/test_iris_knn.py`.
This unit test tests the function `distance_matrix_to_knn_indices` with a small toy example.

### Step 2.3: Mapping neighboud indices to species indices

In Step 2.2 we find the training data points that are closes to the test data points and store the results in an index matrix of shape `[n, k]`, where `n` is the number of test vectors and `k` is the number of neighbours.
To find which species of accurs most frequently in the neighbourhood of each test point, we need to map this index matrix to a species index matrix.
This matrix has the same shape, but contains the corresponding species indices.
This step is already implemented in `IrisData.knn_indices_to_species_indices`. 
You do not need to implement it, but you will need to use this function for one of the later steps.

### Step 2.4: Determine which species is most frequent amongst the k neighbours [10 points]

To determine which species is most frequent amongst the k neighbours, implement the `count_species_index` function. 
This function receives as input a numpy array `species_idxs` of shape `[n, k]`, where each row corresponds to one of the test feature vectors and each column corresponds to one of the k neighbours. 
The entry `[i, j]` of this array `species_idxs` is set to the species index of the jth neighbour of the ith test feature vector.
This function then counts how often each species occurs in the neighbourhood of each test feature vector.
These counts are then returned in an numpy array of shape `[n, 3]`, where the jth column corresponds to how often the jth species occurs in the neighbourhood of a test vector.

The corresponding unit test for this can be found in `test_count_species_index` in the file `test/test_iris_knn.py`.
This unit test tests the function `count_species_index` with a small toy example.
Your implementation must count the number of species for any number of species (i.e. it must execute correctly for any setting of the `num_species` parameter).

To implement this function, you may need to use a for loop.
For full marks, find an implementation such that the number of iterations the for loop executes is minimized.
For full marks, do not use the numpy functions `bincount`, `histogram`, or `digitize`.


### Step 2.5: Implement the final kNN routine [10 points]

To put all these functions and methods together, implement `IrisData.predict_species_knn`.
In your implementation, you should also use the method `IrisData.knn_idices_to_species_indices` which maps indices of training feature vectors to their corresponding species indices.
The implementation should beform the following steps:

1. Compute the distance matrix.
1. Find the indices of the k nearest neigbours in the training feature vectors.
1. Map these indices to species indices.
1. Determine which species is most frequent amongst the k nearest neighbours for each test feature vector.
1. Return an array of shape `[n,]` where each entry corresponds to the predicted species index for each of the `n` test feature vectors.

You can test your implementation using the following Jupyter notebook cell for $k=5$.
The predicted indices are then logged to console.

In [ ]:
knn_pred = iris_data.predict_species_knn(iris_data.test_features, k=5)
display(Markdown(f"Predicted species: {str(knn_pred)}"))

## Step 3: Evaluate classifier performance

### Step 3.1: Compute the accuracy on the test set of the classifier [5 points]

Compute the accuracy on the test set by computing how many test features were correctly labelled.

In [ ]:
# Insert your code here.

### Step 3.2: Compute the confusion matrix [15 points]

In multi-class classification, the confusion matrix encodes the distribution of which class label was predicted by the classifier for each true class label.
In our example, we have three different classes (species) and the confusion matrix has a size of `[3, 3]`.
Each row and each column corresponds to one of the a specific classes (species).
Each entry `[i, j]` counts how often each the class `j` was predicted for test vectors that belong to class `i`.
This means that the diagonal entries count how often a class was correctly predicted (i.e. class `i` was predicted for a test vector that belongs to class `i`).

Implement the confusion matrix calculation in the function `confusion_matrix`.
To debug your implementation, you can use the unit test `test_confusion_matrix`.
You can then run the following cell to visualize your results.

For full marks, only use one for loops that iterate over every cell in the confusion matrix.

In [ ]:
conf_mat = confusion_matrix(knn_pred, iris_data.test_species_idxs)
fig = px.imshow(
    img=conf_mat,
    x=iris_data.species_names,
    y=iris_data.species_names,
    text_auto=True,
    labels=dict(color="Count"),
    color_continuous_scale="Blues"
)
fig.update_layout(
    coloraxis_showscale=False, 
    width=270, 
    height=220, 
    margin={"t": 1, "b": 1, "l": 1, "r": 1},
)
fig.show()

### Step 3.3: Performance summary [10 points]

Which classification errors does kNN make on the Iris dataset?
Which species is always classified correctly and why would that be the case?
You can argue using the scatter plot from Step 1 and the confusion matrix.

[Insert your answer here]